In [2]:
from nuwa_pytorch.nuwa_pytorch import Transformer

In [22]:
transformer = Transformer(dim=128, depth=4, sparse_3dna_attn=True, sparse_3dna_video_shape=[8, 16, 16])

In [23]:
[8, 16, 16][-1]

16

In [5]:
import torch

In [20]:
out = transformer(torch.randn(32, 64, 128))

In [21]:
print(out.size())

torch.Size([32, 64, 128])


In [24]:
from nuwa_pytorch.nuwa_pytorch import Sparse3DNA

In [48]:
sparse_3dna = Sparse3DNA(dim=128, video_shape=[10, 32, 32])

In [49]:
max_frames, fmap_size, _ = sparse_3dna.video_shape

In [50]:
sparse_3dna.heads

8

In [83]:
video = torch.randn(1, 10, 3, 256, 256) # (batch, frames, channels, height, width)
imgs = torch.randn(10, 3, 256, 256) # (batch, channels, height, width)

In [84]:
y = sparse_3dna(imgs)

In [85]:
y.size()

torch.Size([3, 10241, 128])

In [103]:
dim = 512
max_video_frames = 5
image_size = 256
vae_num_layers = 4
fmap_size = image_size // (2 ** vae_num_layers)
video_shape = (max_video_frames, fmap_size, fmap_size)
sparse_3dna_dilation = 1
sparse_3dna_dilations = tuple(range(1, sparse_3dna_dilation + 1)) if not isinstance(sparse_3dna_dilation, (list, tuple)) else sparse_3dna_dilation

In [112]:
from nuwa_pytorch.vqgan_vae import VQGanVAE

from nuwa_pytorch.nuwa_pytorch import Embedding, Transformer

vae = VQGanVAE(
    dim = 64,
    num_layers = 4,
    image_size = 256,
    num_conv_blocks = 2,
    vq_codebook_size = 8192
) 

num_image_tokens = vae.codebook_size

image_embedding = Embedding(num_image_tokens, dim, frac_gradient=0.2)

encoder = Transformer(
            dim = dim,
            depth = 6,
            heads = 8,
            dim_head = 64,
            causal = True,
            cross_attend = True,
            attn_dropout = 0.,
            ff_dropout = 0.,
            ff_chunk_size = None,
            shift_video_tokens = True,
            sparse_3dna_video_shape = video_shape,
            sparse_3dna_attn = True,
            sparse_3dna_kernel_size = 3,
            sparse_3dna_dilations = sparse_3dna_dilations,
            sparse_3dna_query_num_frames_chunk = None,
            sparse_3dna_rel_pos_bias = False
        )

In [100]:
from einops import rearrange, repeat

from nuwa_pytorch.nuwa_pytorch import AxialPositionalEmbedding

In [116]:
dim = 512
max_video_frames = 5
image_size = vae.image_size
fmap_size = image_size // (2 ** vae.num_layers)

# preparing text for the encoder
text = torch.randint(0, 20000, (1, 256))
batch, seq_len, frames, device = *text.shape, video.shape[1], text.device

# preparing image for the encoder
imgs = torch.randn(10, 3, 256, 256) # (batch, channels, height, width)

# preparing video for the encoder
video = torch.randn(1, 10, 3, 256, 256) # (batch, frames, channels, height, width)
video_shape = (max_video_frames, fmap_size, fmap_size)
video_pos_emb = AxialPositionalEmbedding(dim, shape = video_shape)
video_bos = torch.nn.Parameter(torch.randn(dim))

return_loss = False
frame_indices = vae.get_video_indices(video)
frame_indices = rearrange(frame_indices, 'b ... -> b (...)')
frame_indices_input = frame_indices[:, :-1] if return_loss else frame_indices
frame_embeddings = image_embedding(frame_indices_input)
frame_embeddings = video_pos_emb()[:-1] + frame_embeddings

bos = repeat(video_bos, 'd -> b 1 d', b = batch)
frame_embeddings = torch.cat((bos, frame_embeddings), dim = 1)

RuntimeError: The size of tensor a (1279) must match the size of tensor b (2560) at non-singleton dimension 1

In [109]:
video_pos_emb()[:-1].size()

torch.Size([1279, 512])

In [ ]:
frame_outputs = encoder(frame_embeddings)